In [19]:
# ==========================================================================
# ==========================================================================
# ==========================================================================
# Data Download
# Note: As of 2020, the Census API has been somewhat unreliable. We encourage
# everyone to save all their downloads so you don't run into delays while 
# working on your project. Don't rely on the API to download everyday. 
# ==========================================================================
# ==========================================================================
# ==========================================================================

#!/usr/bin/env python
# coding: utf-8

# ==========================================================================
# Import Libraries
# ==========================================================================

import census
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Proj
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format # avoid scientific notation

In [20]:
def filter_FIPS(df):
    if (city_name not in ('Memphis', 'Boston')):
        df = df[df['county'].isin(FIPS)]
    else:
        fips_list = []
        for i in state:
            county = FIPS[i]
            a = list((df['FIPS'][(df['county'].isin(county))&(df['state']==i)]))
            fips_list = fips_list + a
        df = df[df['FIPS'].isin(fips_list)]
    return df


In [21]:
home

'/Users/lopez_e'

In [22]:
home = str(Path.home())
output_path = home+'/Documents/'
# ==========================================================================
# Set API Key
# ==========================================================================

key = '4c26aa6ebbaef54a55d3903212eabbb506ade381' #insert your API key here!
c = census.Census(key)

In [23]:
# Washington State FIPS
state = '53'
# King, Pierce, and Snohomish counties
FIPS = ['033', '053', '061']
# Format sql query
sql_query='state:{} county:*'.format(state)

# ==========================================================================
# Download Raw Data
# ==========================================================================

# Download ACS 2018 5-Year Estimates
# -------------------------------------------------------------------------- 
df_vars_18=['B19001_001E',
            'B19013_001E',
            'B25077_001E',
            'B25077_001M',
            'B25064_001E',
            'B25064_001M',
            'B15003_001E',
            'B15003_022E',
            'B15003_023E',
            'B15003_024E',
            'B15003_025E',
            'B25034_001E',
            'B25034_010E',
            'B25034_011E',
            'B25003_002E',
            'B25003_003E',
            'B25105_001E',
            'B06011_001E']


# Race/Ethnicity Categories
var_str = 'B03002'
var_list = [var_str+'_'+str(i).zfill(3)+'E' for i in range(1,13) ]
df_vars_18 = df_vars_18 + var_list

# Race/Ethnicity Margins of error
var_str = 'B03002'
var_list = [var_str+'_'+str(i).zfill(3)+'M' for i in range(1,13) ]
df_vars_18 = df_vars_18 + var_list

# Income categories - see notes
var_str = 'B19001'
var_list = []
for i in range (1, 18):
    var_list.append(var_str+'_'+str(i).zfill(3)+'E')
df_vars_18 = df_vars_18 + var_list

# Migration - see notes
var_str = 'B07010'
var_list = []
for i in list(range(25,34))+list(range(36, 45))+list(range(47, 56))+list(range(58, 67)):
    var_list.append(var_str+'_'+str(i).zfill(3)+'E')
df_vars_18 = df_vars_18 + var_list


In [24]:
# Run API query for 2018 data
# --------------------------------------------------------------------------
var_dict_acs5 = c.acs5.get(df_vars_18, geo = {'for': 'tract:*',
                                 'in': sql_query}, year=2018)

In [25]:
var_str = 'B03002'
[var_str+'_'+str(i).zfill(3)+'M' for i in range(1,13) ]

['B03002_001M',
 'B03002_002M',
 'B03002_003M',
 'B03002_004M',
 'B03002_005M',
 'B03002_006M',
 'B03002_007M',
 'B03002_008M',
 'B03002_009M',
 'B03002_010M',
 'B03002_011M',
 'B03002_012M']

In [26]:
# Convert and Rename Variables
# -------------------------------------------------------------------------- 

### Converts variables into dataframe and filters only FIPS of interest
city_name='Seattle'

df_vars_18 = pd.DataFrame.from_dict(var_dict_acs5)
df_vars_18['FIPS']=df_vars_18['state']+df_vars_18['county']+df_vars_18['tract']
df_vars_18 = filter_FIPS(df_vars_18)

### Renames variables

df_vars_18 = df_vars_18.rename(columns = {'B03002_001E':'pop_18',
                                          'B03002_001M':'pop_18_moe',
                                          #population not hispanic/latino
                                          'B03002_002E':'pop_nhislat_18',
                                          'B03002_002M':'pop_nhislat_18_moe',
                                          'B03002_003E':'white_18',
                                          'B03002_003M':'white_18_moe',
                                          'B03002_004E':'blk_18',
                                          'B03002_004M':'blk_18_moe',
                                          'B03002_005E':'aian_18',
                                          'B03002_005M':'aian_18_moe',
                                          'B03002_006E':'asian_18',
                                          'B03002_006M':'asian_18_moe',
                                          'B03002_007E':'nhopi_18',
                                          'B03002_007M':'nhopi_18_moe',
                                          'B03002_008E':'sora_18',
                                          'B03002_008M':'sora_18_moe',
                                          'B03002_009E':'two_18',
                                          'B03002_009M':'two_18_moe',
                                          'B03002_010E':'twosora_18',
                                          'B03002_010M':'twosora_18_moe',
                                          'B03002_011E':'twoexc_18',
                                          'B03002_011M':'twoexc_18_moe',
                                          'B03002_012E':'hislat_18',
                                          'B03002_012M':'hislat_18_moe',
                                          'B19001_001E':'hh_18',
                                          'B19013_001E':'hinc_18',
                                          'B25077_001E':'mhval_18',
                                          'B25077_001M':'mhval_18_se',
                                          'B25064_001E':'mrent_18',
                                          'B25064_001M':'mrent_18_se',
                                          'B25003_002E':'ohu_18',
                                          'B25003_003E':'rhu_18',
                                          'B25105_001E':'mmhcosts_18',
                                          'B15003_001E':'total_25_18',
                                          'B15003_022E':'total_25_col_bd_18',
                                          'B15003_023E':'total_25_col_md_18',
                                          'B15003_024E':'total_25_col_pd_18',
                                          'B15003_025E':'total_25_col_phd_18',
                                          'B25034_001E':'tot_units_built_18',
                                          'B25034_010E':'units_40_49_built_18',
                                          'B25034_011E':'units_39_early_built_18',
                                          'B07010_025E':'mov_wc_w_income_18',
                                          'B07010_026E':'mov_wc_9000_18',
                                          'B07010_027E':'mov_wc_15000_18',
                                          'B07010_028E':'mov_wc_25000_18',
                                          'B07010_029E':'mov_wc_35000_18',
                                          'B07010_030E':'mov_wc_50000_18',
                                          'B07010_031E':'mov_wc_65000_18',
                                          'B07010_032E':'mov_wc_75000_18',
                                          'B07010_033E':'mov_wc_76000_more_18',
                                          'B07010_036E':'mov_oc_w_income_18',
                                          'B07010_037E':'mov_oc_9000_18',
                                          'B07010_038E':'mov_oc_15000_18',
                                          'B07010_039E':'mov_oc_25000_18',
                                          'B07010_040E':'mov_oc_35000_18',
                                          'B07010_041E':'mov_oc_50000_18',
                                          'B07010_042E':'mov_oc_65000_18',
                                          'B07010_043E':'mov_oc_75000_18',
                                          'B07010_044E':'mov_oc_76000_more_18',
                                          'B07010_047E':'mov_os_w_income_18',
                                          'B07010_048E':'mov_os_9000_18',
                                          'B07010_049E':'mov_os_15000_18',
                                          'B07010_050E':'mov_os_25000_18',
                                          'B07010_051E':'mov_os_35000_18',
                                          'B07010_052E':'mov_os_50000_18',
                                          'B07010_053E':'mov_os_65000_18',
                                          'B07010_054E':'mov_os_75000_18',
                                          'B07010_055E':'mov_os_76000_more_18',
                                          'B07010_058E':'mov_fa_w_income_18',
                                          'B07010_059E':'mov_fa_9000_18',
                                          'B07010_060E':'mov_fa_15000_18',
                                          'B07010_061E':'mov_fa_25000_18',
                                          'B07010_062E':'mov_fa_35000_18',
                                          'B07010_063E':'mov_fa_50000_18',
                                          'B07010_064E':'mov_fa_65000_18',
                                          'B07010_065E':'mov_fa_75000_18',
                                          'B07010_066E':'mov_fa_76000_more_18',
                                          'B06011_001E':'iinc_18',
                                          'B19001_002E':'I_10000_18',
                                          'B19001_003E':'I_15000_18',
                                          'B19001_004E':'I_20000_18',
                                          'B19001_005E':'I_25000_18',
                                          'B19001_006E':'I_30000_18',
                                          'B19001_007E':'I_35000_18',
                                          'B19001_008E':'I_40000_18',
                                          'B19001_009E':'I_45000_18',
                                          'B19001_010E':'I_50000_18',
                                          'B19001_011E':'I_60000_18',
                                          'B19001_012E':'I_75000_18',
                                          'B19001_013E':'I_100000_18',
                                          'B19001_014E':'I_125000_18',
                                          'B19001_015E':'I_150000_18',
                                          'B19001_016E':'I_200000_18',
                                          'B19001_017E':'I_201000_18'})

In [27]:
# Washington State FIPS
state = '53'
# King, Pierce, and Snohomish counties
FIPS = ['033', '053', '061']
# Format sql query
sql_query='state:{} county:*'.format(state)

# ==========================================================================
# Download Raw Data
# ==========================================================================

# Download ACS 2018 5-Year Estimates
# -------------------------------------------------------------------------- 
df_vars_12=['B19001_001E',
            'B19013_001E',
            'B25077_001E',
            'B25077_001M',
            'B25064_001E',
            'B25064_001M',
            'B15003_001E',
            'B15003_022E',
            'B15003_023E',
            'B15003_024E',
            'B15003_025E',
            'B25034_001E',
            'B25034_010E',
            #'B25034_011E',
            'B25003_002E',
            'B25003_003E',
            'B25105_001E',
            'B06011_001E']


# Race/Ethnicity Categories
var_str = 'B03002'
var_list = [var_str+'_'+str(i).zfill(3)+'E' for i in range(1,13) ]
df_vars_12 = df_vars_12 + var_list

# Race/Ethnicity Margins of error
var_str = 'B03002'
var_list = [var_str+'_'+str(i).zfill(3)+'M' for i in range(1,13) ]
df_vars_12 = df_vars_12 + var_list

# Income categories - see notes
var_str = 'B19001'
var_list = [var_str+'_'+str(i).zfill(3)+'E' for i in range(1,18)]
df_vars_12 = df_vars_12 + var_list

# Migration - see notes
var_str = 'B07010'
var_list = []
for i in list(range(25,34))+list(range(36, 45))+list(range(47, 56))+list(range(58, 67)):
    var_list.append(var_str+'_'+str(i).zfill(3)+'E')
df_vars_12 = df_vars_12 + var_list


In [28]:
# Run API query for 2012 data
# --------------------------------------------------------------------------
var_dict_acs5 = c.acs5.get(df_vars_12, geo = {'for': 'tract:*',
                                 'in': sql_query}, year=2012)

In [29]:
df_vars_12 = pd.DataFrame.from_dict(var_dict_acs5)
df_vars_12['FIPS']=df_vars_12['state']+df_vars_12['county']+df_vars_12['tract']
df_vars_12 = filter_FIPS(df_vars_12)

In [30]:
### Renames variables

df_vars_12 = df_vars_12.rename(columns = {'B03002_001E':'pop_12',
                                          'B03002_001M':'pop_12_moe',
                                          #population not hispanic/latino
                                          'B03002_002E':'pop_nhislat_12',
                                          'B03002_002M':'pop_nhislat_12_moe',
                                          'B03002_003E':'white_12',
                                          'B03002_003M':'white_12_moe',
                                          'B03002_004E':'blk_12',
                                          'B03002_004M':'blk_12_moe',
                                          'B03002_005E':'aian_12',
                                          'B03002_005M':'aian_12_moe',
                                          'B03002_006E':'asian_12',
                                          'B03002_006M':'asian_12_moe',
                                          'B03002_007E':'nhopi_12',
                                          'B03002_007M':'nhopi_12_moe',
                                          'B03002_008E':'sora_12',
                                          'B03002_008M':'sora_12_moe',
                                          'B03002_009E':'two_12',
                                          'B03002_009M':'two_12_moe',
                                          'B03002_010E':'twosora_12',
                                          'B03002_010M':'twosora_12_moe',
                                          'B03002_011E':'twoexc_12',
                                          'B03002_011M':'twoexc_12_moe',
                                          'B03002_012E':'hislat_12',
                                          'B03002_012M':'hislat_12_moe',
                                          'B19001_001E':'hh_12',
                                          'B19013_001E':'hinc_12',
                                          'B25077_001E':'mhval_12',
                                          'B25077_001M':'mhval_12_se',
                                          'B25064_001E':'mrent_12',
                                          'B25064_001M':'mrent_12_se',
                                          'B25003_002E':'ohu_12',
                                          'B25003_003E':'rhu_12',
                                          'B25105_001E':'mmhcosts_12',
                                          'B15003_001E':'total_25_12',
                                          'B15003_022E':'total_25_col_bd_12',
                                          'B15003_023E':'total_25_col_md_12',
                                          'B15003_024E':'total_25_col_pd_12',
                                          'B15003_025E':'total_25_col_phd_12',
                                          'B25034_001E':'tot_units_built_12',
                                          'B25034_010E':'units_40_49_built_12',
                                          'B25034_011E':'units_39_early_built_12',
                                          'B07010_025E':'mov_wc_w_income_12',
                                          'B07010_026E':'mov_wc_9000_12',
                                          'B07010_027E':'mov_wc_15000_12',
                                          'B07010_028E':'mov_wc_25000_12',
                                          'B07010_029E':'mov_wc_35000_12',
                                          'B07010_030E':'mov_wc_50000_12',
                                          'B07010_031E':'mov_wc_65000_12',
                                          'B07010_032E':'mov_wc_75000_12',
                                          'B07010_033E':'mov_wc_76000_more_12',
                                          'B07010_036E':'mov_oc_w_income_12',
                                          'B07010_037E':'mov_oc_9000_12',
                                          'B07010_038E':'mov_oc_15000_12',
                                          'B07010_039E':'mov_oc_25000_12',
                                          'B07010_040E':'mov_oc_35000_12',
                                          'B07010_041E':'mov_oc_50000_12',
                                          'B07010_042E':'mov_oc_65000_12',
                                          'B07010_043E':'mov_oc_75000_12',
                                          'B07010_044E':'mov_oc_76000_more_12',
                                          'B07010_047E':'mov_os_w_income_12',
                                          'B07010_048E':'mov_os_9000_12',
                                          'B07010_049E':'mov_os_15000_12',
                                          'B07010_050E':'mov_os_25000_12',
                                          'B07010_051E':'mov_os_35000_12',
                                          'B07010_052E':'mov_os_50000_12',
                                          'B07010_053E':'mov_os_65000_12',
                                          'B07010_054E':'mov_os_75000_12',
                                          'B07010_055E':'mov_os_76000_more_12',
                                          'B07010_058E':'mov_fa_w_income_12',
                                          'B07010_059E':'mov_fa_9000_12',
                                          'B07010_060E':'mov_fa_15000_12',
                                          'B07010_061E':'mov_fa_25000_12',
                                          'B07010_062E':'mov_fa_35000_12',
                                          'B07010_063E':'mov_fa_50000_12',
                                          'B07010_064E':'mov_fa_65000_12',
                                          'B07010_065E':'mov_fa_75000_12',
                                          'B07010_066E':'mov_fa_76000_more_12',
                                          'B06011_001E':'iinc_12',
                                          'B19001_002E':'I_10000_12',
                                          'B19001_003E':'I_15000_12',
                                          'B19001_004E':'I_20000_12',
                                          'B19001_005E':'I_25000_12',
                                          'B19001_006E':'I_30000_12',
                                          'B19001_007E':'I_35000_12',
                                          'B19001_008E':'I_40000_12',
                                          'B19001_009E':'I_45000_12',
                                          'B19001_010E':'I_50000_12',
                                          'B19001_011E':'I_60000_12',
                                          'B19001_012E':'I_75000_12',
                                          'B19001_013E':'I_100000_12',
                                          'B19001_014E':'I_125000_12',
                                          'B19001_015E':'I_150000_12',
                                          'B19001_016E':'I_200000_12',
                                          'B19001_017E':'I_201000_12'})

In [31]:
# ==========================================================================
# Export Files
# ==========================================================================
# Note: ouput paths can be altered by changing the 'output path variable above'

# Merge 2012 & 2018 files - same geometry
df_vars_summ = df_vars_18.merge(df_vars_12, on ='FIPS')

In [32]:
from pathlib import Path
df_vars_summ.to_csv(output_path.replace(" ", "")+'census_summ_2018.csv')